In [1]:
import pandas as pd
import nltk
import faiss
import torch
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
def clean_text(text):
    text = text.lower()
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\C","")
    text = re.sub(r"\[.*?\]", "", text) 
    return text.strip()

<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\glenn\AppData\Local\Temp\ipykernel_29892\583663975.py:3: SyntaxWarning: invalid escape sequence '\C'
  text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\C","")


In [3]:
def process_txt_files(folder_path):
    df = pd.read_csv(folder_path)
    df["Entrevista"] = df["Entrevista"].apply(clean_text)  
    return df

In [12]:
folder_path = "../CorpusRI.csv"
df = process_txt_files(folder_path)

In [5]:
# 1. Cargar el CSV y fragmentar las entrevistas en oraciones
def load_and_split_csv(file_path):
    df = pd.read_csv(file_path)
    df = process_txt_files(folder_path)
    nltk.download('punkt')
    df['Oraciones'] = df['Entrevista'].apply(nltk.sent_tokenize)
    return df


In [6]:
# 2. Generar embeddings con sentence-transformers
def generate_embeddings(sentences, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences, convert_to_tensor=True, normalize_embeddings=True)
    return embeddings


In [7]:
# 3. Indexar en FAISS
def create_faiss_index(embeddings):
    d = embeddings.shape[1]  # Dimensión de los embeddings
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.cpu().numpy())
    return index

In [8]:
# 4. Recuperar oraciones relevantes
def retrieve_similar_sentences(query, model, index, sentences, k=5):
    query_embedding = model.encode([query], convert_to_tensor=True, normalize_embeddings=True).cpu().numpy()
    distances, indices = index.search(query_embedding, k)
    return [sentences[i] for i in indices[0]]


In [9]:
# 5. Generación de texto con Mistral-7B

def generate_response(context, query, model_name='mistralai/Mistral-7B-Instruct-v0.1'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map='auto')
    
    prompt = f"Pregunta: {query}\nContexto relevante: {context}\nRespuesta: "
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [10]:
folder_path = r"C:\Users\glenn\Documents\Repositories\RI-Project-2B\CorpusRI.csv"  # Ruta al archivo CSV
query = "¿Que opina Leonidas Iza respecto a la delincuencia?"
df = load_and_split_csv(folder_path)
sentences = [s for sublist in df['Oraciones'] for s in sublist]  # Lista plana de oraciones

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = generate_embeddings(sentences, model_name='sentence-transformers/all-MiniLM-L6-v2')

index = create_faiss_index(embeddings)
relevant_sentences = retrieve_similar_sentences(query, embedding_model, index, sentences)
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\glenn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
response = generate_response(' '.join(relevant_sentences), q<uery)
print("Respuesta Generada:\n", response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Respuesta Generada:
 Pregunta: ¿Que opina Leonidas Iza respecto a la delincuencia?
Contexto relevante: ¿qué opina el respecto? leonidas, usted qué opina del voto nulo? pero es un gusto compartir esta mañana con el ingeniero leonidas iza. algo más que acotar a esta breve descripción de su persona, leonidas. dicen que está fracturada, por ejemplo, con el señor leonidas isa.
Respuesta: ¡Hola! ¿Qué puedo hacer para ayudarte hoy?
